*This notebook gets reverse-dns entries from `ra_one_off_extensions` table, and searches the sample words to identify dsl and cable deploymentss and create a temporary file `temp.sql` with UPDATE queries that can be run on `ra_one_off_extensions` table*
- - - 

In [19]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [20]:
import sqlite3
import pandas as pd
import requests
import ipaddress
import time

In [21]:
DB_LOCATION = 'lastmile.db'
RA_ONE_OFF_EXTENSIONS = 'ra_one_off_extensions'
RA_PROBE_API = 'ra_probe_api'
RA_ASN = 'ra_asn'

In [22]:
d_fav_rdns = {

     'CABLE'     :     ['hsd1', 'cable', 'hfc']
   , 'DSL'       :     ['adsl', 'dsl', 'dslb', 'xdsl', 'luxdsl']
   , 'FIBRE'     :     ['fios', 'ftth', 'fiber7', 'fttb']    
}

In [23]:
con = sqlite3.connect(DB_LOCATION)

In [24]:
query = '''SELECT    asn_holder_name
           FROM      %s
        '''%(RA_ASN)

df1 = pd.read_sql(query, con)

In [25]:
holder_names = []
for value in df1.values: holder_names.append(value[0])

In [26]:
query = '''SELECT    ai.probeid, a.asn_access_type, oe.reverse_dns, '' as access_technology
           FROM      %s as oe, %s as ai, %s as a
           WHERE     oe.probeid = ai.probeid
           AND       ai.asn_v4 =  a.asn
        '''%(RA_ONE_OFF_EXTENSIONS, RA_PROBE_API, RA_ASN)

df = pd.read_sql(query, con)

In [27]:
df.head()

,probeid,asn_access_type,reverse_dns,access_technology
0,3,DSL,idante.xs4all.nl,
1,4,DSL,soenda.xs4all.nl,
2,5,DSL,a83-163-239-181.adsl.xs4all.nl,
3,7,CABLE,541C2103.cm-5-5a.dynamic.ziggo.nl,
4,8,DSL,dfk.xs4all.nl,


In [28]:
def create_set_of_tlds():
    try: res = requests.get('http://www.internic.net/domain/root.zone')
    except Exception as e: print(e)
    else: data = res.text
        
    tld_list = []
    for line in data.split('\n'):
        try: tld = line.split()[0]
        except Exception as e: print('.')
        else:
            tld = tld.rstrip('.')
            tld_list.append(tld)

    return set(tld_list)    

In [29]:
tlds = create_set_of_tlds()

.


In [30]:
df = df.dropna()

In [31]:
import re
def split_rdns_by_word(rdns):
    try: rdns_list = re.split('\.|-',rdns)
    except Exception as e: print(rdns, type(e)); return None
    else: return rdns_list
df['rdns_list'] = df['reverse_dns'].apply(split_rdns_by_word)

In [32]:
d = {}
for i, row in df.iterrows():
    probeid = row['probeid']
    word_list = row['rdns_list']

    for word in word_list: 
        if word in tlds: continue
        try: int(word)
        except:
            try: d[word].append(probeid)
            except KeyError as e: d[word] = []; d[word].append(probeid)

In [33]:
from collections import OrderedDict
ordered_d = OrderedDict(sorted(d.items(), key=lambda x: len(x[1]), reverse=True))

In [34]:
for k, v in ordered_d.items():
    if len(v) >= 200: print('%s: %d'%(k, len(v)))

c: 244
comcast: 232
hsd1: 232
dynamic: 207


In [35]:
for i, row in df.iterrows():
    probeid = row['probeid']; rdns_list = row['rdns_list']
    for k,v in d_fav_rdns.items():
        v_set = set(v); rdns_set = set(rdns_list)     
        intersect = v_set.intersection(rdns_set)
        if len(intersect) > 0: 
            df.ix[i, 'access_technology'] = k
            break

In [36]:
df.head()

,probeid,asn_access_type,reverse_dns,access_technology,rdns_list
0,3,DSL,idante.xs4all.nl,,"[idante, xs4all, nl]"
1,4,DSL,soenda.xs4all.nl,,"[soenda, xs4all, nl]"
2,5,DSL,a83-163-239-181.adsl.xs4all.nl,DSL,"[a83, 163, 239, 181, adsl, xs4all, nl]"
3,7,CABLE,541C2103.cm-5-5a.dynamic.ziggo.nl,,"[541C2103, cm, 5, 5a, dynamic, ziggo, nl]"
4,8,DSL,dfk.xs4all.nl,,"[dfk, xs4all, nl]"


In [37]:
del df['asn_access_type']
del df['reverse_dns']
del df['rdns_list']

In [38]:
df.head()

,probeid,access_technology
0,3,
1,4,
2,5,DSL
3,7,
4,8,


In [39]:
fsock = open('temp.sql', 'w')
fsock.write('BEGIN TRANSACTION;\n')

for i, row in df.iterrows():
    probeid = row['probeid']; at = row['access_technology']
    if at == '': continue
    query = 'UPDATE %s SET access_type_technology = "%s" WHERE probeid = %d;'%(RA_ONE_OFF_EXTENSIONS, at, probeid)
    fsock.write('%s\n'%query)
    
    
fsock.write('END TRANSACTION;\n')
fsock.close()